In [1]:
from bs4 import BeautifulSoup

In [2]:
import urllib.request

In [3]:
from urllib.parse import quote

In [4]:
import pandas as pd

https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=" + str(mode) + "&target=after

In [5]:
def get_movie_reviews(mcode, page_num=10):
    movie_review_df = pd.DataFrame(columns=('Title','Score'))
    url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=" + str(mcode) + "&target=after"
    idx = 0

    for _ in range(0, page_num):
        movie_page = urllib.request.urlopen(url).read()
        movie_page_soup = BeautifulSoup(movie_page,'html.parser')
        review_list = movie_page_soup.find_all('td', {'class':'title'})

        for review in review_list:
            title = review.find('a',{'class':'movie color_b'}).get_text()
            score = review.find('em').get_text()
            movie_review_df.loc[idx] = [title, score]
            idx += 1
            print('#', end='')

        try:
            url = "https://movie.naver.com" + movie_page_soup.find('a', {'class':'pg_next'}).get('href')
#             print(url)
        except:
            break
        
    return movie_review_df

In [6]:
movie_review_df = get_movie_reviews(18847, 10)
movie_review_df

####################################################################################################

,Title,Score
0,타이타닉,10
1,타이타닉,10
2,타이타닉,10
3,타이타닉,10
4,타이타닉,10
...,...,...
95,타이타닉,10
96,타이타닉,10
97,타이타닉,10
98,타이타닉,10


In [ ]:
url = "https://movie.naver.com/movie/point/af/list.naver"
naver_movie = urllib.request.urlopen(url).read()